1. База данных для хранения исходных данных, промежуточных и окончательных результатов.
Создано в DB Browser for SQLite:

CREATE TABLE "Sentences" (
	"ID"	INTEGER,
	"Sentence"	TEXT,
	"Metadata"	TEXT,
	PRIMARY KEY("ID" AUTOINCREMENT)
)
CREATE TABLE "Words" (
	"ID"	INTEGER,
	"Token"	TEXT,
	"Lemma"	TEXT,
	"POS"	TEXT,
	"ID_sent"	INTEGER,
	PRIMARY KEY("ID" AUTOINCREMENT)
)

In [2]:
import sqlite3
import pandas as pd

In [24]:
conn = sqlite3.connect('DoctorWho.db')
cur = conn.cursor()

Для коллекции предложений connlu идёт первичная итерация по предложениям sent с метадатой meta, вторичная итерация по токенам token с леммой lemma и тегом tag

dicts = [[{'id': 1, 'text': 'Test', 'upos': 'NOUN', 'xpos': 'NN', 'feats': 'Number=Sing', 'misc': 'start_char=0|end_char=4'}, {'id': 2, 'text': 'sentence', 'upos': 'NOUN', 'xpos': 'NN', 'feats': 'Number=Sing', 'misc': 'start_char=5|end_char=13'}, {'id': 3, 'text': '.', 'upos': 'PUNCT', 'xpos': '.', 'misc': 'start_char=13|end_char=14'}]]

In [ ]:
# пишем данные в базу
id_sent = 1
id_token = 1
for line in connlu:
    # meta = 
    # sent = 
    cur.execute(
            """
            INSERT INTO Sentences
                VALUES (?,?,?)
            """, (id_sent,sent,meta) # id, предложение, метадата
    conn.commit()
    # for word in sent:
        # token =
        # lemma = 
        # tag = 
        cur.execute(
            """
            INSERT INTO Words
                VALUES (?,?,?,?)
            """, (id_token,token,lemma,tag,id_sent) # id, токен, лемма, тег, id предложения, из которго взято слово
        conn.commit()
        id_token +=1
     id_sent += 1

In [2]:
# request = [([entry], search_type)], search_type IN token, lemma, POS, (lemma,POS)
# Если нужна последовательность из 2-3 слов
# Вынимаем из базы  ID-шники предложений, в которых встречаются искомые вхождения и ID-шники самих вхождений
def search_sequence(request):
    i = 1
    output = []
    for entry in request:
        if entry[1] != 'lemma+pos':
            query = 'SELECT ID_sent, ID FROM Words WHERE Words.'+ entry[1]+ '='  + entry[0][1]
        else:
            query = 'SELECT ID_sent, ID FROM Words WHERE Words.'+ entry[1][0]+ '='  + entry[0][0] + 'AND Words.'+ entry[1][1]+ '=' + entry[0][1]
        i +=1
        res = cur.execute(query)
        res.fetchall()
        output.append(res)
    return(output)

In [3]:
# request = [([entry], search_type)], search_type IN token, lemma, POS, (lemma,POS)
# Если нужно одно слово - сразу вынимаем из базы предложения и метадату
def search_one(request):
    if request[0][1] != 'lemma+pos':
        query = 'SELECT Sentence, Metadata FROM Sentences JOIN Words ON Words.ID_sent = Sentences.ID WHERE Words.' + request[1][0]+ '=' + request[0][0]
    else:
        query = 'SELECT Sentence, Metadata FROM Sentences JOIN Words ON Words.ID_sent = Sentences.ID WHERE Words.' + request[1][0]+ '=' + request[0][0]+ 'AND Words.' + request[1][1]+ '=' + request[0][1]
    res = cur.execute(query)
    res.fetchall()
    return(res)

In [4]:
# request = [([entry], search_type)], search_type IN token, lemma, POS, (lemma,POS)
if len(request) == 1:
    output = search_one(request)
else:
    output = search_sequence(request)

NameError: name 'request' is not defined

In [ ]:
res = cur.execute("SELECT score FROM movie")

res.fetchall()
[(8.2,), (7.5,)]

In [26]:
sentences_by_token

,Sentence,Metadata
0,проверочное предложение,"Доктор Кто, фанфик 1"


In [7]:
# поиск по словоформе requested_token
token_query = """
        SELECT
            Sentence,
            Metadata
        FROM Sentences
            JOIN Words ON Words.ID_sent = Sentences.ID
        WHERE
            Words.token = ?
        """
sentences_by_token = pd.read_sql_query(token_query, params = [requested_token], con=conn)

In [20]:
lemma_pos_request

,Sentence,Metadata
0,проверочное предложение,"Доктор Кто, фанфик 1"


In [9]:
# поиск по лемме requested_lemma
requested_lemma = 'проверочный'
lemma_query = """
        SELECT
            Sentence,
            Metadata
        FROM Sentences
            JOIN Words ON Words.ID_sent = Sentences.ID
        WHERE
            Words.lemma = ?
        """
lemma_request = pd.read_sql_query(lemma_query, params = [requested_lemma], con=conn)

In [11]:
# поиск по тегу requested_pos
requested_pos = 'ADJ'
pos_query = """
        SELECT
            Sentence,
            Metadata
        FROM Sentences
            JOIN Words ON Words.ID_sent = Sentences.ID
        WHERE
            Words.POS = ?
        """
pos_request = pd.read_sql_query(pos_query, params = [requested_pos], con=conn)

In [19]:
# поиск по тегу requested_pos, лемме requested_lemma
requested_lemma = 'проверочный'
requested_pos = 'ADJ'
lemma_pos_query = """
        SELECT
            Sentence,
            Metadata
        FROM Sentences
            JOIN Words ON Words.ID_sent = Sentences.ID
        WHERE
            Words.lemma = ? AND Words.POS = ?  
        """
lemma_pos_request = pd.read_sql_query(lemma_pos_query, params = [requested_lemma, requested_pos], con=conn)

In [29]:
conn.close()